In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BaseModelConfigEntity:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))

        # create_directories([self.config.artifacts_root])
        create_directories([self.config.artifacts_root])

    def get_base_model_config(self) -> BaseModelConfigEntity:
        config = self.config.base_model

        create_directories([config.root_dir])

        base_model_config = BaseModelConfigEntity(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return base_model_config

In [4]:
import urllib.request as requests
import tensorflow as tf

[2024-02-05 14:08:18,718: WARNING : module_wrapper: From c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [5]:
class PrepareBaseModel:
    def __init__(self, config: BaseModelConfigEntity):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )
        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units = classes,
            activation = "softmax"
        )(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs=prediction
        )
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes = self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        

In [6]:
try:
    config=ConfigurationManager()
    base_model_config = config.get_base_model_config()
    prep_base_model = PrepareBaseModel(config=base_model_config)
    prep_base_model.get_base_model()
    prep_base_model.update_base_model()
except Exception as e:
    raise e

[2024-02-05 14:08:19,755: INFO : common: yaml file: config\config.yaml loaded sucessfully!]
[2024-02-05 14:08:19,759: INFO : common: yaml file: params.yaml loaded sucessfully!]
[2024-02-05 14:08:19,761: INFO : common: Created directory at: artifacts]
[2024-02-05 14:08:19,762: INFO : common: Created directory at: artifacts/prepare_base_model]
[2024-02-05 14:08:20,077: WARNING : module_wrapper: From c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2024-02-05 14:08:20,260: WARNING : module_wrapper: From c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.
]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape          